# Principal Component Analysis

In [1]:
import hail as hl
hl.init(spark_conf={'spark.driver.memory': '10g'}, tmp_dir='/home/olavur/tmp')

Running on Apache Spark version 2.4.1
SparkUI available at http://hms-beagle-848846b477-48ks9:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/fargen-1-exome/notebooks/qc/hail-20210512-1305-0.2.61-3c86d3ba497a.log


In [2]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from bokeh.models.scales import LogScale
output_notebook()

Loading BokehJS ...

In [36]:
import pandas as pd

## Load FarGen data

Use LD pruned diallelic sites.

In [124]:
BASE_DIR = '/home/olavur/experiments/2020-11-13_fargen1_exome_analysis'
mt = hl.read_matrix_table(BASE_DIR + '/data/mt/ld_pruned_diallelic_common.mt')

In [125]:
n_variants, n_samples = mt.count()
print('Number of variants: ' + str(n_variants))
print('Number of samples: ' + str(n_samples))

Number of variants: 134452
Number of samples: 474


### Add population filters

Use a HWE filter and a minor allele frequency filter. The LD pruned data is already filtered with MAF > 0.01, but we will use a more stringent filter.

In [126]:
mt = mt.annotate_rows(hwe=hl.agg.hardy_weinberg_test(mt.GT))
mt = mt.filter_rows(mt.hwe.p_value > 1e-6)

Calculate allele frequencies.

In [127]:
# The number of alleles at the site is the sum of the ploidy at each site.
# This number should be twice the number of samples.
# If there are missing genotype calls, the number of alleles will be less.
AN_exprs = hl.agg.sum(mt.GT.ploidy)
mt = mt.annotate_rows(AN=AN_exprs)

# Calculate the number of alternate alleles at each site.
AC_exprs = hl.agg.sum(mt.GT.n_alt_alleles())
mt = mt.annotate_rows(AC=AC_exprs)

# Calculate the alternate allele frequency.
mt = mt.annotate_rows(AF=mt.AC / mt.AN)

**TODO:** I can increase the MAF threshold if still a lot of variants remain.

Remove variants with minor allele frequency under 0.05.

In [128]:
maf_filter = 0.05
mt = mt.filter_rows((mt.AF > maf_filter) & (mt.AF < (1 - maf_filter)))

## Annotate birth place region

In [136]:
fargen_rin_ht = hl.import_table(BASE_DIR + '/data/metadata/birthplace/fargen_rin_samplename.csv', delimiter=',')
fargen_rin_ht = fargen_rin_ht.key_by(fargen_rin_ht.rin)

2021-05-12 16:06:26 Hail: INFO: Reading table without type imputation
  Loading field 'rin' as type str (not specified)
  Loading field 'sample' as type str (not specified)


In [137]:
fargen_rin_ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'rin': str 
    'sample': str 
----------------------------------------
Key: ['rin']
----------------------------------------


In [138]:
rin_birthplace_ht = hl.import_table(BASE_DIR + '/data/metadata/birthplace/rin_region.csv', delimiter=',')
# Rename "ind" to "rin".
# Convert the region variable to float.
rin_birthplace_ht = rin_birthplace_ht.transmute(rin=rin_birthplace_ht.ind, birthplace=hl.float64(rin_birthplace_ht.region))

rin_birthplace_ht = rin_birthplace_ht.key_by(rin_birthplace_ht.rin)

2021-05-12 16:06:28 Hail: INFO: Reading table without type imputation
  Loading field 'ind' as type str (not specified)
  Loading field 'region' as type str (not specified)


In [139]:
rin_birthplace_ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'rin': str 
    'birthplace': float64 
----------------------------------------
Key: ['rin']
----------------------------------------


In [140]:
# Annotate the table with the birthplace by the samplenames.
samplename_birthplace_ht = rin_birthplace_ht.annotate(samplename=fargen_rin_ht[rin_birthplace_ht.rin].sample)
samplename_birthplace_ht = samplename_birthplace_ht.key_by(samplename_birthplace_ht.samplename)

In [141]:
samplename_birthplace_ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'rin': str 
    'birthplace': float64 
    'samplename': str 
----------------------------------------
Key: ['samplename']
----------------------------------------


In [143]:
mt = mt.annotate_cols(birthplace = samplename_birthplace_ht[mt.s].birthplace)

In [144]:
p = hl.plot.histogram(mt.birthplace)
show(p)

2021-05-12 16:07:18 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-12 16:07:18 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-12 16:07:19 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-12 16:07:20 Hail: INFO: Ordering unsorted dataset with network shuffle


## Filter related individuals

Estimate the relatedness between the samples by the PC-Relate method, with a minimum alternate allele frequency of 0.001.

In [145]:
pc_rel = hl.pc_relate(mt.GT, 0.001, k=2, statistics='kin')

2021-05-12 16:07:27 Hail: INFO: hwe_normalized_pca: running PCA using 46925 variants.
2021-05-12 16:07:30 Hail: INFO: pca: running PCA with 2 components...
2021-05-12 16:07:51 Hail: INFO: Wrote all 12 blocks of 46925 x 474 matrix with block size 4096.


Plot all the relatedness coefficients in a histogram to get an overview.

In [146]:
p = hl.plot.histogram(pc_rel.kin, title='Histogram of kinship coefficient')
show(p)

2021-05-12 16:07:51 Hail: INFO: wrote matrix with 3 rows and 46925 columns as 12 blocks of size 4096 to /home/olavur/tmp/pcrelate-write-read-81UQFFcEfgnn1AoOQS85Ej.bm
2021-05-12 16:07:51 Hail: INFO: wrote matrix with 46925 rows and 474 columns as 12 blocks of size 4096 to /home/olavur/tmp/pcrelate-write-read-pgHDldXZeZOpIR8C8vs2ml.bm
2021-05-12 16:08:17 Hail: INFO: wrote matrix with 474 rows and 474 columns as 1 block of size 4096 to /home/olavur/tmp/pcrelate-write-read-4jGag1x4r1BIB8FkiMP3k2.bm
2021-05-12 16:08:45 Hail: INFO: wrote matrix with 474 rows and 474 columns as 1 block of size 4096 to /home/olavur/tmp/pcrelate-write-read-oqm90OJKAi1SFQLbshDKui.bm
2021-05-12 16:08:45 Hail: INFO: wrote matrix with 474 rows and 474 columns as 1 block of size 4096 to /home/olavur/tmp/pcrelate-write-read-ZUT0ZzgcKCsiHqkhflRNkF.bm
2021-05-12 16:08:45 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-12 16:08:46 Hail: INFO: wrote matrix with 3 rows and 46925 columns as 12 blocks of

In [147]:
pairs = pc_rel.filter(pc_rel['kin'] > 2**(-3))

Then we find the maximal independent set, consistent of the samples to remove.

In [148]:
related_samples_to_remove = hl.maximal_independent_set(pairs.i, pairs.j, keep=False)

2021-05-12 16:09:40 Hail: INFO: wrote matrix with 3 rows and 46925 columns as 12 blocks of size 4096 to /home/olavur/tmp/pcrelate-write-read-aSTl5nTKM7y2P1JRfKfXfi.bm
2021-05-12 16:09:41 Hail: INFO: wrote matrix with 46925 rows and 474 columns as 12 blocks of size 4096 to /home/olavur/tmp/pcrelate-write-read-6EON74HlMLF1bsIUU4GGQT.bm
2021-05-12 16:10:08 Hail: INFO: wrote matrix with 474 rows and 474 columns as 1 block of size 4096 to /home/olavur/tmp/pcrelate-write-read-QEW1kns1w6OjiK6GctFfvj.bm
2021-05-12 16:10:33 Hail: INFO: wrote matrix with 474 rows and 474 columns as 1 block of size 4096 to /home/olavur/tmp/pcrelate-write-read-41HnlTah2NdkT66JxjOKvq.bm
2021-05-12 16:10:33 Hail: INFO: wrote matrix with 474 rows and 474 columns as 1 block of size 4096 to /home/olavur/tmp/pcrelate-write-read-ZHvD5tGPQv3IN74il5xsBx.bm
2021-05-12 16:10:34 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-12 16:10:34 Hail: INFO: wrote table with 70 rows in 1 partition to /home/olavur/tm

Now we filter these individuals from the matrix table.

In [149]:
pruned_mt = mt.filter_cols(hl.is_defined(related_samples_to_remove[mt.col_key]), keep=False)

In [150]:
samples_before_prune = mt.count_cols()
samples_after_prune = pruned_mt.count_cols()
print('Samples before prune: {n}\nSample after prune: {m}'.format(n=samples_before_prune, m=samples_after_prune))

2021-05-12 16:10:35 Hail: INFO: Coerced sorted dataset
2021-05-12 16:10:35 Hail: INFO: Ordering unsorted dataset with network shuffle


Samples before prune: 474
Sample after prune: 418


## Compute PCA

In [151]:
eigenvalues, scores, loadings = hl.hwe_normalized_pca(pruned_mt.GT, k=2)

2021-05-12 16:10:36 Hail: INFO: Coerced sorted dataset
2021-05-12 16:10:36 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-12 16:10:39 Hail: INFO: hwe_normalized_pca: running PCA using 46925 variants.
2021-05-12 16:10:39 Hail: INFO: Coerced sorted dataset
2021-05-12 16:10:39 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-12 16:10:41 Hail: INFO: pca: running PCA with 2 components...


In [152]:
pruned_mt = pruned_mt.annotate_cols(scores = scores[pruned_mt.s].scores)

In [153]:
p = hl.plot.scatter(pruned_mt.scores[0],
                    pruned_mt.scores[1],
                    label=hl.str(pruned_mt.birthplace),
                    title='PCA', xlabel='PC1', ylabel='PC2')
p.plot_width = 800
p.plot_height = 600
show(p)

2021-05-12 16:10:48 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-12 16:10:48 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-12 16:10:49 Hail: INFO: Coerced sorted dataset
2021-05-12 16:10:49 Hail: INFO: Ordering unsorted dataset with network shuffle


## Filter outliers

In the PCA plot above, it seems we have a few outliers. These individuals may have non-Faroese ancestry, whether they are part Faroese or not, or they may be very closely related. Either way, they prevent us from detecting possible population structure in the data.

As we see in the plot above, both PC 1 and 2 describe the variation between these outliers and the rest of the samples. So if we remove these outlier individuals, we may be able to detect population structure.

In [154]:
no_outerliers_mt = pruned_mt.filter_cols(pruned_mt.scores[1] > -0.2)

Now we just do the PCA again, and see if PC 1 and 2 show signs of population structure.

In [155]:
eigenvalues, scores, loadings = hl.hwe_normalized_pca(no_outerliers_mt.GT, k=2)

2021-05-12 16:12:50 Hail: INFO: Coerced sorted dataset
2021-05-12 16:12:50 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-12 16:12:52 Hail: INFO: hwe_normalized_pca: running PCA using 46925 variants.
2021-05-12 16:12:52 Hail: INFO: Coerced sorted dataset
2021-05-12 16:12:52 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-12 16:12:54 Hail: INFO: pca: running PCA with 2 components...


In [156]:
no_outerliers_mt = no_outerliers_mt.annotate_cols(scores = scores[no_outerliers_mt.s].scores)

In [157]:
p = hl.plot.scatter(no_outerliers_mt.scores[0],
                    no_outerliers_mt.scores[1],
                    label=hl.str(no_outerliers_mt.birthplace),
                    title='PCA', xlabel='PC1', ylabel='PC2')
p.plot_width = 800
p.plot_height = 600
show(p)

2021-05-12 16:13:05 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-12 16:13:05 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-12 16:13:06 Hail: INFO: Coerced sorted dataset
2021-05-12 16:13:06 Hail: INFO: Ordering unsorted dataset with network shuffle


From this PCA, it looks like there is very little population structure. But I'm not so convinced, I see this as a failure to detect population structure, not a success to show that there is no population structure.